In [ ]:
df1 = pd.DataFrame({'count': df['category_title'].value_counts(),
                    'sum':df.groupby('category_title')['transaction_amount'].sum(), 
                    'mean': df.groupby('category_title')['transaction_amount'].mean(),
                   'std': df.groupby('category_title')['transaction_amount'].std(),
                  'min': df.groupby(['category_title'])['transaction_amount'].min(),
                  'max' : df.groupby(['category_title'])['transaction_amount'].max(),
                  'q25' : df.groupby(['category_title'])['transaction_amount'].quantile(0.25),
                  'q75' : df.groupby(['category_title'])['transaction_amount'].quantile(0.75)}
                 )

In [ ]:
df1['iqr'] = df1['q75'] - df1['q25']
df1['cut_off'] =  df1['iqr']*1.5
df1['lower'] = df1['q25'] - df1['cut_off']
df1['upper'] = df1['q75'] + df1['cut_off']

In [ ]:
temp1 = df.loc[:,['category_title', 'transaction_amount']]

In [ ]:
outlier = temp1.loc[(temp1['category_title']=='프로그래밍') & 
              (temp1['transaction_amount'] > df1.loc['프로그래밍', 'upper'])& 
              (temp1['transaction_amount'] < df1.loc['프로그래밍', 'lower']) , 'sale'].index

In [ ]:
for i in ['교육', '기타', '데이터사이언스', '디자인', '마케팅', '부동산/금융', '업무 생산성', '영상/3D',
       '크리에이티브', '투자/재테크', '프로그래밍']:
    outlier = temp1.loc[(temp1['category_title']==i) & 
              (temp1['transaction_amount'] > df1.loc[i, 'upper'])| 
              (temp1['transaction_amount'] < df1.loc[i, 'lower']) , 'transaction_amount'].index
    temp1.drop(outlier, inplace = True)
temp1

In [ ]:
df2 = pd.DataFrame({'count': temp1['category_title'].value_counts(),
                    'sum':temp1.groupby('category_title')['transaction_amount'].sum(), 
                    'mean': temp1.groupby('category_title')['transaction_amount'].mean(),
                   'std': temp1.groupby('category_title')['transaction_amount'].std(),
                  'min': temp1.groupby(['category_title'])['transaction_amount'].min(),
                  'max' : temp1.groupby(['category_title'])['transaction_amount'].max(),
                  'q25' : temp1.groupby(['category_title'])['transaction_amount'].quantile(0.25),
                  'q75' : temp1.groupby(['category_title'])['transaction_amount'].quantile(0.75)}
                 )

In [ ]:
df2['iqr'] = df2['q75'] - df2['q25']
df2['cut_off'] =  df2['iqr']*1.5
df2['lower'] = df2['q25'] - df2['cut_off']
df2['upper'] = df2['q75'] + df2['cut_off']

In [ ]:
df2['category'] = ['교육', '기타', '데이터사이언스', '디자인', '마케팅', '부동산/금융', '업무 생산성', '영상/3D',
       '크리에이티브', '투자/재테크', '프로그래밍']
df1['category'] = ['교육', '기타', '데이터사이언스', '디자인', '마케팅', '부동산/금융', '업무 생산성', '영상/3D',
       '크리에이티브', '투자/재테크', '프로그래밍']
df2 = df2.sort_values('sum', ascending = False)
df2

In [ ]:
df2['outlier_count'] = df1['count'] - df2['count']
df2['outlier_sum'] = df1['sum'] - df2['sum']

In [ ]:
df2.loc[:,['outlier_count','outlier_sum']]

In [ ]:
temp2 = df2.loc[:, ['sum', 'outlier_sum']]

In [ ]:
temp2['diff'] = temp2['sum'] - temp2['outlier_sum']
temp2['portion'] = (temp2['outlier_sum']) / (temp2['sum']+temp2['outlier_sum'])*100

In [ ]:
temp3 = df[df['type'] == 'REFUND'].groupby(by = 'category')['transaction'].agg(sum)

In [ ]:
df2['Refund'] = abs(temp3)
df1['Refund'] = abs(temp3)

In [ ]:
#1) 카테고리별 매출액 순위 
plt.figure(figsize = [10,5])
df2 = df2.sort_values('sum', ascending = False)
sns.barplot(data=df2, x='category',y = 'sum')

# => 주요 수입원은 -- 이다. 

In [ ]:
# 위에 대한 근거자료 // 전체 매출에서 각 카테고리의 비중
plt.pie(df2['sum'], labels = df2['category'], autopct='%.1f%%', startangle = 260,
       counterclock = False, shadow = True)
plt.show()

In [ ]:
# 카테고리별 수량 대비 매출액
df2['perSale'] = df2['sum'] / df2['count']
df2['perSale'].plot(kind = 'barh', y = 'category')

In [ ]:
# 카테고리별 리펀드 비율 비교
df2['Refund_rate'] = df2['Refund'] / df2['sum'] *100
df2['Refund_rate']

In [ ]:
plt.figure(figsize = [10,5])
df2 = df1.sort_values('sum', ascending = False) 
df2 = df2.sort_values('sum', ascending = False)
sns.barplot(data=df2, x='category',y = 'Refund_rate')

# => 환불 비율이 높은 마케팅과 크리에이티브, 또한 환불 총액이 높은 프로그래밍과 데이터사이언스에 대해
#    환불을 줄일 만한 대책이 필요

In [ ]:
#3) Turkey Fence 방식을 사용한 각 카테고리 아웃라이어 비중
sns.set_theme(style="whitegrid")
f, ax = plt.subplots(figsize=(6, 15))
sns.set_color_codes("pastel")
sns.barplot(x="sum", y="category", data=df2,
            label="category", color="b")
sns.set_color_codes("muted")
sns.barplot(x="outlier_sum", y="category", data=df2,
            label="category", color="b")

#ax.legend(ncol=2, loc="lower right", frameon=True)
#ax.set(xlim=(0, 24), ylabel="",
#       xlabel="Automobile collisions per billion miles")
sns.despine(left=True, bottom=True)
